# 1. 什麼是 Agent?
在生成式人工智慧 (Generative AI) 當中，Agent 指的是由語言模型來當代理人，自動為你執行一連串的推理 (reasoning) 或動作 (action) 的架構。這邊的動作包括查詢網路、呼叫其他應用的 API、執行程式碼等等。

跟 Chain 不一樣的是，雖然 Chain 也可以執行一連串的生成 (generation) 和動作 (action) 但它的執行路徑都是寫死的，而 Agent 則是可以根據得到的觀察 (observation) 和反饋 (feedback) 來自行決定下一步要做什麼事。Agent 相比 Chain 更靈活但同時也更不可測。

以下我們將會打造一個問答用的 Agent，並檢視其從問題到得出答案的一連串步驟，比較和 Chain 有什麼不同

# 2. 什麼是 ReAct 架構
下面我們會使用採取 ReAct 架構的 Agent 作為例子。([ReAct: Synergizing Reasoning and Acting in Language Models](https://python.langchain.com/docs/modules/agents/agent_types/react_docstore))

而 ReAct 架構可以粗淺地用如下的演算法解釋
```
for step in (1...MAX_STEPS):
    THOUGHT, ACTION_or_ANSWER = Plan(QUESTION, HISTORY)
    if "Final Answer:" in ACTION_or_ANSWER:
        ANSWER = ACTION_or_ANSWER
        return ANSWER
    else:
        ACTION = ACTION_or_ANSWER
        OBSERVATION = Execute(ACTION)
        HISTORY += (THOUGHT, ACTION, OBSERVATION)
```
可以看到這個演算法有幾個變數 (variable)
- QUESTION: 使用者問題
- THOUGHT: 語言模型認為下一步要做什麼的敘述
- Action: 動作。可以是使用搜尋引擎或執行程式，包含執行動作所需的輸入 (例如搜尋關鍵字或程式碼)
- HISTORY: 過去的想法跟動作的紀錄
- ANSWER: 最終的答案

和兩個函式
- `Plan`: 由語言模型決定下一步要做什麼和怎麼做
- `Execute`: 實際去執行該動作

用一個假的例子來體驗看看

In [1]:
def plan(quesiton, history):
    match STEP:
        case 1:
            return "我應該先判斷它的氣味", "{'function': '聞', 'argument': '鼻子'}"
        case 2:
            return "我應該嘗試點燃它", "{'function': '點燃', 'argument': '打火機'}"
        case 3:
            return "我知道最後的答案了", "Final Answer: 這不是白開水而是酒!"
        
def execute(action):
    action = eval(action)
    match STEP:
        case 1:
            return "有股酒味"
        case 2:
            return "本該是白開水的東西燒起來了"

question = "學長遞了杯白開水給我，我應該喝嗎？"
print(f"Question: {question}")
history = []
for STEP in range(1,10):
    print(f"步驟{STEP}=================================================")
    thought, action_or_answer = plan(question, history)
    if 'Final Answer:' in action_or_answer:
        answer = action_or_answer
        print(answer)
        break
    else:
        action = action_or_answer
        observation = execute(action)
        print(f"Thought: {thought}")
        print(f"Action: {action}")
        print(f"Observation: {observation}")
        history.append((thought, action, observation))

Question: 學長遞了杯白開水給我，我應該喝嗎？
步驟1=================================================
Thought: 我應該先判斷它的氣味
Action: {'function': '聞', 'argument': '鼻子'}
Observation: 有股酒味
步驟2=================================================
Thought: 我應該嘗試點燃它
Action: {'function': '點燃', 'argument': '打火機'}
Observation: 本該是白開水的東西燒起來了
步驟3=================================================
Final Answer: 這不是白開水而是酒!


# 2. 建立 ReAct Agent

接下來我們要實際地來實踐 ReAct 這個架構

In [2]:
from langchain import hub
from langchain.agents import load_tools
from langchain.tools.render import render_text_description
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor

from langchain_setup import OpenAI, ChatOpenAI, tracing_v2_enabled_if_api_key_set

## 2.1 打造 Planning Step
我們從上面可以看到 ReAct Agent 架構裡最核心的就是那個 `Plan` 的功能，而這個功能的實作其實就是一個最基本的 Chain (Prompt + Model + Output parser)。

### 2.1.1 Prompt

首先我們看到 Prompt 的部分，我們直接從網路共享的 langchain hub 從抓取，並檢視其內容

In [3]:
prompt = hub.pull("hwchase17/react-json")
for message_prompt in prompt.messages:
    role = message_prompt.__class__.__name__.replace('MessagePromptTemplate', '')
    print(f"[{role}]: {message_prompt.prompt.template}")

[System]: Answer the following questions as best you can. You have access to the following tools:

{tools}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: {tool_names}

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action:
```
$JSON_BLOB
```
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to always use the exact characters `Final A

這個 Prompt 有幾個重點：
1. 可以看到這個 Prompt 「預告」了接下來的生成 (generate) 內容，每行的開頭應該是 Question -> (Thougth, Action) x N 次 -> Thought, Final Answer, 並解釋了每個開頭後面應該要生成什麼內容。
2. 除此之外它也指示了語言模型在生成 Action 後面的內容是必須要按照 JSON 格式。

還有數個 Placeholders
1. `tools` 和 `tool_names`: 告訴語言模型有哪些工具可用，這些工具的輸入形式 (input schema)
2. `input`: 使用者的問題
3. `agent_scratchpad`: 過去紀錄 (history)

### 2.1.2 Tools

這邊我們使用抓取兩個 Langchain 提供的現成工具，duckduckgo search 、基於語言模型的數學計算器，來提供給我們的 Agent 使用。

In [4]:
tools = load_tools(["ddg-search", "llm-math"], llm=OpenAI()) # 因 llm-math 有用到所以傳入一個語言模型

事先填滿 Prompt 的 `tools` 和 `tool_names` placeholders，可以看到其敘述了工具的性質和使用方法。

In [5]:
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
print(render_text_description(tools))

duckduckgo_search: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.


### 2.1.3 語言模型
為了讓語言模型產生完 Action 後可以等我們實際執行完 Action 再回來繼續，我們設定如果語言模型生成 (generate) "\nObeservation" 時即停止生成

In [6]:
chat_model_with_stop = ChatOpenAI(temperature=0).bind(stop=["\nObservation"])

### 2.1.4 串接

In [7]:
plan = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

### 2.1.5 創建 Agent

In [8]:
agent_executor = AgentExecutor(agent=plan, tools=tools, verbose=True)

# 3. 實際使用看看

In [9]:
with tracing_v2_enabled_if_api_key_set(project_name='tutorial'):
    outputs = agent_executor.invoke(
        {
            "input": "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?"
        }
    )
    print(outputs)



> Entering new AgentExecutor chain...
Question: Who is Leo DiCaprio's girlfriend?
Thought: I can search for this information using DuckDuckGo Search.
Action:
```
{
  "action": "duckduckgo_search",
  "action_input": "Leo DiCaprio girlfriend"
}
```Thirty-seven-year-old Leo and 23-year-old Erin dated for almost a year, and the split was drama-free. E! News actually asked Erin about it at an event and she said, "I having the best time right ... Who Is Leonardo DiCaprio's Girlfriend Vittoria Ceretti? The Italy native is a model and made her debut at Elite Model Look, a prestigious international modeling competition, at the age of 14 ... Back in 2018, his ex-girlfriend Camila Morrone made her LACMA gala debut after being with DiCaprio for nearly a year. Here is the photo below of her debut: LOS ANGELES, CA - NOVEMBER 03: Cami ... But alongside his work, Leonardo DiCaprio's personal life has been the subject of constant scrutiny. Documented most succinctly in this now infamous chart, all of